# 简化版SHAP分析Notebook

本notebook将进行简化的SHAP分析，包括：
1. 源数据检查
2. 动量因子计算
3. 模型训练和SHAP分析
4. 异常信号监测

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 导入配置
import config

print("环境准备完成")

环境准备完成


## 1. 源数据检查

首先加载并检查源数据的质量和结构。

In [2]:
def load_and_sanity_check_data(index_name='csi800'):
    """
    加载并检查源数据
    :param index_name: 指数名称
    :return: DataFrame
    """
    print("=== 源数据检查 ===")
    file_path = f'data/{index_name}_daily_returns.csv'
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    print(f"数据形状: {df.shape}")
    print(f"日期范围: {df['date'].min()} 到 {df['date'].max()}")
    print(f"缺失值统计:\n{df.isnull().sum()}")
    
    return df

# 加载并检查数据
df = load_and_sanity_check_data(config.INDEX_NAME)

=== 源数据检查 ===
数据形状: (2371, 2)
日期范围: 2015-12-01 00:00:00 到 2025-08-29 00:00:00
缺失值统计:
date      0
return    1
dtype: int64


## 2. 动量因子计算

计算三个动量信号：短期、中期、长期。

In [3]:
def calculate_momentum_signals(df, short_window=5, medium_window=66, long_window=252):
    """
    计算三个动量信号
    :param df: 包含日期和收益率的DataFrame
    :param short_window: 短期窗口大小
    :param medium_window: 中期窗口大小
    :param long_window: 长期窗口大小
    :return: 包含动量信号的DataFrame
    """
    print("=== 动量因子计算 ===")
    
    # 短期动量
    short_term_return = df['return'].rolling(window=short_window).sum()
    short_term_volatility = df['return'].rolling(window=short_window).std()
    df['short_momentum'] = short_term_return / short_term_volatility
    
    # 中期动量
    medium_term_return = df['return'].rolling(window=medium_window).sum()
    medium_term_volatility = df['return'].rolling(window=medium_window).std()
    df['medium_momentum'] = medium_term_return / medium_term_volatility
    
    # 长期动量
    long_term_return = df['return'].rolling(window=long_window).sum()
    long_term_volatility = df['return'].rolling(window=long_window).std()
    df['long_momentum'] = long_term_return / long_term_volatility
    
    return df

# 计算动量因子
df = calculate_momentum_signals(df)

=== 动量因子计算 ===


## 3. 模型训练和SHAP分析

训练模型并进行SHAP分析。

In [4]:
def create_target_variable(df, future_days=5):
    """
    创建目标变量
    :param df: 包含收益率的DataFrame
    :param future_days: 未来天数
    :return: 包含目标变量的DataFrame
    """
    df['future_return'] = df['return'].shift(-future_days).rolling(window=future_days).sum()
    return df

def prepare_features_and_target(df):
    """
    准备特征和目标变量
    :param df: 包含所有数据的DataFrame
    :return: 特征和目标变量
    """
    # 删除包含NaN的行
    df = df.dropna()
    
    # 选择特征
    features = ['short_momentum', 'medium_momentum', 'long_momentum']
    X = df[features]
    y = df['future_return']
    
    return X, y, df

def train_model_and_analyze_shap(X, y):
    """
    训练模型并进行SHAP分析
    :param X: 特征
    :param y: 目标变量
    :return: 训练好的模型和SHAP值
    """
    print("=== 模型训练和SHAP分析 ===")
    # 分割训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 训练随机森林模型
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # 使用SHAP计算所有样本的贡献度
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    return model, shap_values

# 创建目标变量
df = create_target_variable(df, config.FUTURE_DAYS)

# 准备特征和目标变量
X, y, df_processed = prepare_features_and_target(df)

# 训练模型并进行SHAP分析
model, shap_values = train_model_and_analyze_shap(X, y)

=== 模型训练和SHAP分析 ===


## 4. 异常信号监测

使用SHAP来监测异常信号。

In [5]:
def detect_abnormal_signals(shap_values, df_with_dates):
    """
    检测异常信号
    :param shap_values: SHAP贡献度
    :param df_with_dates: 包含日期的DataFrame
    :return: None
    """
    print("=== 检测异常信号 ===")
    
    # 创建DataFrame存储贡献度
    contributions_df = pd.DataFrame(shap_values, columns=['短期动量', '中期动量', '长期动量'])
    contributions_df['date'] = df_with_dates['date'].iloc[:len(shap_values)].values
    
    # 计算每个动量因子的1%分位数和99%分位数
    percentiles = [1, 99]
    
    for column in ['短期动量', '中期动量', '长期动量']:
        values = contributions_df[column]
        lower_percentile = np.percentile(values, percentiles[0])
        upper_percentile = np.percentile(values, percentiles[1])
        
        # 找出低于1%分位数和高于99%分位数的极端值
        lower_extremes = contributions_df[contributions_df[column] < lower_percentile]
        upper_extremes = contributions_df[contributions_df[column] > upper_percentile]
        
        # 合并极端值
        extremes = pd.concat([lower_extremes, upper_extremes])
        
        print(f"\n{column}的异常信号检测:")
        print(f"1%分位数: {lower_percentile:.6f}")
        print(f"99%分位数: {upper_percentile:.6f}")
        print(f"异常信号数量: {len(extremes)}")

# 检测异常信号
detect_abnormal_signals(shap_values, df_processed)

=== 检测异常信号 ===

短期动量的异常信号检测:
1%分位数: -0.013353
99%分位数: 0.015151
异常信号数量: 44

中期动量的异常信号检测:
1%分位数: -0.017716
99%分位数: 0.019616
异常信号数量: 44

长期动量的异常信号检测:
1%分位数: -0.027731
99%分位数: 0.026665
异常信号数量: 44
